# DNA weaver

In [1]:
! pip install dnaweaver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.0/873.0 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: tinycss2
    Found existing installation: tinycss2 1.3.0
    Uninstalling tinycss2-1.3.0:
      Successfully uninstalled tinycss2-1.3.0


In [5]:
import dnaweaver as dw
from dnaweaver import (
DnaAssemblyStation
)

sequence = 'ATGACGACGCCGTGGTCGGGAGAGTTAATTGTGGATTTATTAAATCAGCGTCCTCGCGGACAAGTCGTGTTAGATGCTGCTAATGGGGTGGATTATGAGCTGCGCTCTCAAGAGACCGGACCAATTGACATCGACGCACACGGACCCGCTCGCTGCCTGACCGGAACGGGCTCCCGTACGACGCGTTGGATTCGCCAACATCAGCGCAATACTCGCGTTTTGGGCCTGTCGCATGaccctacagttgctgacgcgcgcgaattgttacgtttaggggctttggGCCTTTTACCTAAAAATGCCGAGCCAGCGGAAATCTTCCAAGCGTTAGATCAAGCTGCCGCCGACCACATTCACACGAATGACCTTGTGCGCGACGCACCCCGTACCCCCGCTCTTCCTGAAGAAGAGGAGGCAGAACGCGTCAACGCTTTGCCAGACCAGGGCGCGGCGCGCACGGGCGCTTTGGGAGCCCAAGGTTGGCCCCACCATACTCCTGACCAACCGGCGCGCTTAGGT'
quote = assembly_station.get_quote(sequence, with_assembly_plan=True)
assembly_plan_report = quote.to_assembly_plan_report()
assembly_plan_report.write_full_report("report.zip")


-----

WeasyPrint could not import some external libraries. Please carefully follow the installation steps before reporting an issue:
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#installation
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#troubleshooting 

-----



OSError: cannot load library 'libgobject-2.0-0': dlopen(libgobject-2.0-0, 0x0002): tried: 'libgobject-2.0-0' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibgobject-2.0-0' (no such file), '/Users/arianemora/miniconda3/envs/oligo/bin/../lib/libgobject-2.0-0' (no such file), '/usr/lib/libgobject-2.0-0' (no such file, not in dyld cache), 'libgobject-2.0-0' (no such file).  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libgobject-2.0-0'

In [6]:
! ls  /opt/homebrew/opt/glib/lib/

girepository-1.0            libgmodule-2.0.0.dylib
glib-2.0                    libgmodule-2.0.a
libgio-2.0.0.dylib          libgmodule-2.0.dylib
libgio-2.0.a                libgobject-2.0.0.dylib
libgio-2.0.dylib            libgobject-2.0.a
libgirepository-2.0.0.dylib libgobject-2.0.dylib
libgirepository-2.0.a       libgthread-2.0.0.dylib
libgirepository-2.0.dylib   libgthread-2.0.a
libglib-2.0.0.dylib         libgthread-2.0.dylib
libglib-2.0.a               pkgconfig
libglib-2.0.dylib


In [4]:
! pip install PyGObject

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [79 lines of output]
           ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/661.5 kB ? eta -:--:--
           ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.5/661.5 kB 5.8 MB/s eta 0:00:00
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Installing backend dependencies: started
        Installing backend dependencies: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        × Prepari